In [1]:
import os

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [221]:
import re
import nltk
import string
#nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1131)>


In [222]:
MACCROBAT_data_dir = "../data/MACCROBAT"

In [332]:
import sys
sys.path.append('../scripts')  # add the parent directory to the search path

from ann_to_bio_conversion import AnnToBioConverter

In [342]:
from importlib import reload

import ann_to_bio_conversion
reload(ann_to_bio_conversion)
from ann_to_bio_conversion import AnnToBioConverter

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1131)>


In [343]:
ann_bio = AnnToBioConverter(input_dir=MACCROBAT_data_dir,output_dir="../data/BIO_FILES")
#                             filtered_entities=[   
#     'Biological_attribute',    'Disease_disorder',    'Sign_symptom',    'Medication',    'Therapeutic_procedure',    'Diagnostic_procedure',    'Lab_value',    'Personal_background',    'Family_history',    'Age',    'Sex',    'Outcome',    'History'])

In [344]:
ann_bio.load_data(hyphen_split=True)
ann_bio.write_bio_files()

[2, 3, 4, 5]

In [328]:
del a[2]
a

[2, 3, 5]

In [ ]:
# Initialize the text and annotations
text = "This is an example sentence."
annotations = [{'label': 'Label1', 'start': 5, 'end': 7},
               {'label': 'Label2', 'start': 5, 'end': 7},
               {'label': 'Label3', 'start': 8, 'end': 15}]

# Initialize the tags
tags = ['O'] * len(text)

# Sort the annotations based on their start positions
sorted_annotations = sorted(annotations, key=lambda ann: ann['start'])

# Set the corresponding positions in the tag list for each annotation
for ann in sorted_annotations:
    
    label = ann['label']
    start = ann['start']
    end = ann['end']
    
    for i in range(start, end):
        
        if tags[i] == 'O':
            # Set the start position of the annotation
            tags[i] = f'B-{label}'
            
        elif tags[i].startswith('B-'):
            # Set intermediate positions of the annotation
            tags[i] += f';I-{label}'
            
        else:
            # This token is part of another named entity, so we need to start a new one
            tags[i] = f'B-{label}'

# Write the tags to a .bio file
with open('text.bio', 'w') as f:
    for i, char in enumerate(text):
        tag = tags[i]
        f.write(f"{char}\t{tag}\n")
    f.write('\n')


In [32]:
"test test ajay ajay".find("ajay", 10)

10

In [43]:
import re

text = "Hello-World!\u200aThis-is a ! test."
words = [match.group(0) for match in re.finditer(r'[^\s\u200a-]+', text)]
indices = [(match.start(), match.end()) for match in re.finditer(r'[^\s\u200a-]+', text)]

print(words)
print(indices)


['Hello', 'World!', 'This', 'is', 'a', '!', 'test.']
[(0, 5), (6, 12), (13, 17), (18, 20), (21, 22), (23, 24), (25, 30)]


In [46]:
text = "Hello-World!\u200aThis-is ! a test."
words = [match.group(0) for match in re.finditer(r'[^\s\u200a-]+', text)]
processed_words = list(map(remove_trailing_punctuation, words))
indices = [(match.start(), match.start() + len(token)) for match, token in zip(re.finditer(r'[^\s\u200a-]+', text), processed_words)]

print(processed_words)
print(indices)


['Hello', 'World', 'This', 'is', '', 'a', 'test']
[(0, 5), (6, 11), (13, 17), (18, 20), (21, 21), (23, 24), (25, 29)]


In [42]:
import re

text = "Hello\u200aWorld! This-is a test."
words = [match.group(0) for match in re.finditer(r'\S+', text)]
indices = [(match.start(), match.end()) for match in re.finditer(r'\S+', text)]

print(words)
print(indices)

['Hello', 'World!', 'This-is', 'a', 'test.']
[(0, 5), (6, 12), (13, 20), (21, 22), (23, 28)]


In [261]:
def remove_trailing_punctuation(token):
    while token and (re.search(r'[^\w\s\']', token[-1]) or (len(token) > 1 and token[-1] == "'")):
        token = token[:-1]
    return token

def split_text(file_id):
    text = data[file_id]['text']
    regex_match = r'[^\s\u200a\-\u2010-\u2015\u2212\uff0d]+' # r'[^\s\u200a\-\—\–]+'
    tokens = []

    words = [match.group(0) for match in re.finditer(regex_match, text)]
    processed_words = list(map(remove_trailing_punctuation, words))
    indices = [(match.start(), match.start() + len(token)) for match, token in zip(re.finditer(regex_match, text), processed_words)]
    
    return processed_words, indices

In [194]:

def splitting_tokens(file_id, start, end, hyphen_split):
    """
    Splits a multi-word token into separate tokens and returns a list of tokens and their respective start and end indices.
    :param file_id: (str) The ID of the file containing the token.
    :param start: (int) The starting index of the token in the text.
    :param end: (int) The ending index of the token in the text.
    :return: (tuple) A tuple containing a list of tokens and their respective start and end indices.
    """

    text = data[file_id]['text']
    token = text[start:end]

    extra_sep = ['\u200a']
    if hyphen_split:
        extra_sep += ['-', '\u2010', '\u2011', '\u2012', '\u2013', '\u2014', '\u2015', '\u2212', '\uff0d']

    new_range = []
    tokens = []

    curr = start
    new_start = None

    for c in token + " ":
        if c.isspace() or c in extra_sep:
            if new_start:
                new_range.append([new_start, curr])
                tokens.append(text[new_start:curr])
                new_start = None
        elif not new_start:
            new_start = curr
        curr += 1

    return tokens, new_range
splitting_tokens("28103924", 0, 5000, True)

(['ur',
  'patient',
  'was',
  'a',
  '7',
  'year',
  'old',
  'Italian',
  'boy',
  'born',
  'after',
  'an',
  'uneventful',
  'gestation',
  'of',
  'normal',
  'duration.',
  'At',
  'the',
  'age',
  'of',
  '16',
  'months,',
  'he',
  'presented',
  'with',
  'a',
  'clinically',
  'evident',
  'enlarged',
  'abdomen',
  'and',
  'was',
  'referred',
  'for',
  'oncological',
  'examination.',
  'Initial',
  'tests',
  'revealed',
  'anemia,',
  'thrombocytopenia,',
  'and',
  'splenomegaly.',
  'A',
  'bone',
  'marrow',
  'biopsy',
  'revealed',
  'the',
  'presence',
  'of',
  'foam',
  'cells,',
  'which',
  'led',
  'to',
  'suspicion',
  'of',
  'lysosomal',
  'storage',
  'disease.',
  'Biochemical',
  'testing',
  'revealed',
  'elevated',
  'level',
  'of',
  'acid',
  'phosphatase',
  '(47.8',
  'IU/L',
  '[normal',
  'range',
  '5',
  '7',
  'IU/L])',
  'and',
  'chitotriosidase',
  'activity',
  '(508',
  'nmol/mg',
  'protein',
  '[normal',
  'range',
  '5.9',
  

0 Our
1 patient
2 was
3 a
4 7
5 year
6 old
7 Italian
8 boy
9 born
10 after
11 an
12 uneventful
13 gestation
14 of
15 normal
16 duration
17 At
18 the
19 age
20 of
21 16
22 months
23 he
24 presented
25 with
26 a
27 clinically
28 evident
29 enlarged
30 abdomen
31 and
32 was
33 referred
34 for
35 oncological
36 examination
37 Initial
38 tests
39 revealed
40 anemia
41 thrombocytopenia
42 and
43 splenomegaly
44 A
45 bone
46 marrow
47 biopsy
48 revealed
49 the
50 presence
51 of
52 foam
53 cells
54 which
55 led
56 to
57 suspicion
58 of
59 lysosomal
60 storage
61 disease
62 Biochemical
63 testing
64 revealed
65 elevated
66 level
67 of
68 acid
69 phosphatase
70 (47.8
71 IU/L
72 [normal
73 range
74 5
75 7
76 IU/L
77 and
78 chitotriosidase
79 activity
80 (508
81 nmol/mg
82 protein
83 [normal
84 range
85 5.9
86 41.0
87 nmol/mg
88 protein
89 as
90 well
91 as
92 reduced
93 beta
94 glucosidase
95 activity
96 (2
97 nmol/mg/protein
98 [normal
99 range
100 4.5
101 18.0
102 nmol/mg/protein
103 Molecular
1

In [286]:
import math
def remove_trailing_punctuation(token):
    while token and (re.search(r'[^\w\s\']', token[-1]) or (len(token) > 1 and token[-1] == "'")):
        token = token[:-1]
    return token

def split_text(file_id):
    text = data[file_id]['text']
    regex_match = r'[^\s\u200a\-\u2010-\u2015\u2212\uff0d]+' # r'[^\s\u200a\-\—\–]+'
    
    tokens = []
    start_end_ranges = []

    sentence_breaks = []
    
    start_idx = 0
    
    for sentence in text.split('\n'):
        words = [match.group(0) for match in re.finditer(regex_match, sentence)]
        processed_words = list(map(remove_trailing_punctuation, words))
        sentence_indices = [(match.start(), match.start() + len(token)) for match, token in zip(re.finditer(regex_match, sentence), processed_words)]
        
        # Update the indices to account for the current sentence's position in the entire text
        sentence_indices = [(start_idx + start, start_idx + end) for start, end in sentence_indices]

        start_end_ranges.extend(sentence_indices)
        tokens.extend(processed_words)
        
        sentence_breaks.append(len(tokens))

        start_idx += len(sentence) + 1
    return tokens, start_end_ranges, sentence_breaks


In [275]:
text = data['19860925']['text']
len(text.split('\n')[0]), text[170:]

(169,
 'During holidays in his home country, this Cuban patient suffered from a cold with fever and a strong cough.\nThe strong dry cough persisted after recovery from the cold.\nThe patient did not report any loss of weight.\nThe initial CT scan of the thorax showed a 12 × 4 cm solid mass paravertebral right in the lower thorax without any signs of metastases (Figure 1).\nThe bronchoscopy (Figure \u200b2) with non-bleeding biopsy revealed a mass of the lower right bronchus which histologically and immunohistologically provided evidence of a granular cell or Abrikossoff tumor [1].\nThe bronchial lavage which followed was negative for malignant cells.\nThe patient was discharged and surgical intervention was planned.\nFour days after discharge a spontaneous hemothorax developed.\nThe patient needed to be readmitted and the hemothorax was drained.\nNo malignant cells were detected in the cytological examination of the drained liquid.\nAfter an uneventful course and decreasing of the hema

In [277]:
split_text('19860925')

(['Our',
  '24',
  'year',
  'old',
  'non',
  'smoking',
  'male',
  'patient',
  'presented',
  'with',
  'repeated',
  'hemoptysis',
  'in',
  'May',
  '2008',
  'with',
  '4',
  'days',
  'of',
  'concomitant',
  'right',
  'thoracic',
  'pain',
  'which',
  'intensified',
  'while',
  'breathing',
  'During',
  'holidays',
  'in',
  'his',
  'home',
  'country',
  'this',
  'Cuban',
  'patient',
  'suffered',
  'from',
  'a',
  'cold',
  'with',
  'fever',
  'and',
  'a',
  'strong',
  'cough',
  'The',
  'strong',
  'dry',
  'cough',
  'persisted',
  'after',
  'recovery',
  'from',
  'the',
  'cold',
  'The',
  'patient',
  'did',
  'not',
  'report',
  'any',
  'loss',
  'of',
  'weight',
  'The',
  'initial',
  'CT',
  'scan',
  'of',
  'the',
  'thorax',
  'showed',
  'a',
  '12',
  '',
  '4',
  'cm',
  'solid',
  'mass',
  'paravertebral',
  'right',
  'in',
  'the',
  'lower',
  'thorax',
  'without',
  'any',
  'signs',
  'of',
  'metastases',
  '(Figure',
  '1',
  'The',


In [296]:
def tag_token(tags, token_pos, tag):
    
    if token_pos > 0 and f'{tag}' in tags[token_pos-1]:
            
        if tags[token_pos] == 'O':
            tags[token_pos] = f'I-{tag}'
        else:
            tags[token_pos] += f'I-{tag}'
    else:
        if tags[token_pos] == 'O':
            tags[token_pos] = f'B-{tag}'
        else:
            tags[token_pos] += f'B-{tag}'


for file_id in data:
    text = data[file_id]['text']
    annotations = data[file_id]['annotations']
    
    # Tokenizing
    tokens, token2text, sentence_breaks = split_text(file_id)
    
    # Initialize the tags
    tags = ['O'] * len(tokens)
    
    ann_pos = 0
    token_pos = 0

    while ann_pos < len(annotations) and token_pos < len(tokens):
        
        label = annotations[ann_pos]['label']
        start = annotations[ann_pos]['start']
        end = annotations[ann_pos]['end']
        
        ann_word = text[start:end]
        
        # find the next word that fall between the annotation start and end
        while token_pos < len(tokens) and token2text[token_pos][0] < start:
            token_pos += 1
        
        if tokens[token_pos] == ann_word or  \
            ann_word in tokens[token_pos] or \
            re.sub(r'\W+', '', ann_word) in re.sub(r'\W+', '', tokens[token_pos]):
            tag_token(tags, token_pos, label)
        elif ann_word in tokens[token_pos-1] or \
             ann_word in tokens[token_pos-1] or \
             re.sub(r'\W+', '', ann_word) in re.sub(r'\W+', '', tokens[token_pos-1]):
            tag_token(tags, token_pos-1, label)
        else:
            print(tokens[token_pos], tokens[token_pos-1], ann_word, label)
            
        # increment to access next annotation
        ann_pos += 1
        
    # Write the tags to a .bio file
    with open(f"../data/BIO_FILES/{file_id}.bio", 'w') as f:
        for i in range(len(tokens)):
            token = tokens[i].strip()
            if token:
                if i in sentence_breaks:
                    f.write("\n")
                f.write(f"{tokens[i]}\t{tags[i]}\n")


[13,
 21,
 50,
 81,
 101,
 114,
 125,
 137,
 146,
 155,
 165,
 169,
 180,
 211,
 220,
 240,
 285,
 292,
 305,
 320,
 354,
 363,
 408,
 414,
 440,
 451,
 466,
 466]

In [239]:



words = []
for file_id in data:
    text = data[file_id]['text']
    annotations = new_d[file_id]['annotations']
    
    # Tokenizing
    tokens, token2text = split_text(file_id)
    
    # Initialize the tags
    tags = ['O'] * len(tokens)
    
    ann_pos = 0
    token_pos = 0
    count = 0
    while ann_pos < len(annotations) and token_pos < len(tokens):
        
        label = annotations[ann_pos]['label']
        start = annotations[ann_pos]['start']
        end = annotations[ann_pos]['end']
        
        # find the next word that fall between the annotation start and end
        while token_pos < len(tokens) and token2text[token_pos][0] < start:
            token_pos += 1
        if tokens[token_pos] == remove_trailing_punctuation(text[start:end]):
            count += 1
            words.append((tokens[token_pos], remove_trailing_punctuation(text[start:end])))
#             print(tokens[token_pos], text[start:end], file_id, start, end, token2text[token_pos])
#         elif tokens[token_pos-1].replace("(", "").startswith(remove_trailing_punctuation(text[start:end])):
        elif text[start:end] in tokens[token_pos-1]:
            token_pos -= 1
            count += 1
            words.append((tokens[token_pos-1].replace("(", ""), remove_trailing_punctuation(text[start:end])))
        else:
            print(f"curr {tokens[token_pos]} {token2text[token_pos]} {file_id} ann_Pos is {ann_pos} {start}, {end}, {text[start:end]}")
            print(f"prev {tokens[token_pos-1]} {token2text[token_pos-1]}")
        # increment to access next annotation
        ann_pos += 1
    print(count, len(annotations))


121 121
248 248
223 223
curr 6 (2620, 2621) 28103924 ann_Pos is 181 2620, 2624, 6–12
prev every (2614, 2619)
242 243
curr and (1796, 1799) 27064109 ann_Pos is 173 1790, 1793, 60%
prev (60 (1789, 1792)
curr examination (3246, 3257) 27064109 ann_Pos is 293 3246, 3256, examinatio
prev Histological (3233, 3245)
330 332
curr 18 (914, 916) 20146086 ann_Pos is 73 914, 919, 18–21
prev exons (908, 913)
253 254
156 156
179 179
curr Epstein (4006, 4013) 28353588 ann_Pos is 359 4006, 4018, Epstein–Barr
prev no (4003, 4005)
385 386
214 214
curr D2 (3092, 3094) 28272235 ann_Pos is 287 3092, 3097, D2–40
prev for (3088, 3091)
curr D2 (3130, 3132) 28272235 ann_Pos is 290 3130, 3135, D2–40
prev for (3126, 3129)
303 305
curr AVED.4 (1946, 1952) 23242090 ann_Pos is 214 1946, 1950, AVED
prev in (1943, 1945)
curr movements.5 (2258, 2269) 23242090 ann_Pos is 249 2258, 2267, movements
prev involuntary (2246, 2257)
327 329
curr arthritis/tenosynovitis (142, 165) 23124805 ann_Pos is 8 142, 151, arthritis
prev w

In [194]:
file_ids = []
for file in os.listdir(MACCROBAT_data_dir):
    file_id = file.split(".")[0]
    if file_id not in file_ids:
        file_ids.append(file_id)

In [195]:
import os

txt_dir = MACCROBAT_data_dir
ann_dir = MACCROBAT_data_dir

## Step 1: Load the .txt files

In [196]:
data = {}
for file_name in os.listdir(txt_dir):
    if file_name.endswith(".txt"):
        with open(os.path.join(txt_dir, file_name), "r") as f:
            text = f.read()
        file_id = file_name.split('.')[0]
        data[file_id] = {
                "text": text, 
                "annotations": []
        }

## Step 2: Load the .ann files and extract entity labels

In [197]:
def splitting_tokens(text, start, end):

    token = text[start:end]
    new_range = []
    tokens = []
    curr = start
    new_start = None
    for c in token + " ":
        if c.isspace() or c == '\u200a':
            if new_start:
                new_range.append([new_start, curr])
                tokens.append(text[new_start:curr])
                new_start = None
        elif not new_start:
            new_start = curr
        curr += 1
    
    return tokens, new_range


In [220]:
for file_name in os.listdir(ann_dir):

    if file_name.endswith(".ann"):
        with open(os.path.join(ann_dir, file_name), "r") as f:
                    
            file_id = file_name.split('.')[0]
            start_end_ranges = []
            annotations = []

            for line in f:
                if line.startswith("T"):
                    fields = line.strip().split("\t")
                    if len(fields[1].split(" ")) > 1:
                        label = fields[1].split(" ")[0]
                        ## Extracting start end indices (Few annotations contain more than one disjoint ranges) 
                        start_end_range = [list(map(int, start_end.split())) for start_end in ' '.join(fields[1].split(" ")[1:]).split(';')]
                        # start_end_ranges.append(start_end_range)
                        ## start_end_ranges - list of list [[start_1, end_1], [start_2, end_2],  ...]
                        ## few start_end contains more than one token. need to split 
                        
                        start_end_range_fixed = []
                        for start, end in start_end_range:
                            text = data[file_id]['text']
                            tokens, start_end_split_list = splitting_tokens(text, start, end)
                            start_end_range_fixed.extend(start_end_split_list)
                        
                        ## Adding labels, start, end to annotations
                        for start, end in start_end_range_fixed:
                            annotations.append({
                                "label": label,
                                "start": start,
                                "end": end
                            })
            # sort annotations based on 'start' key before adding it to our dataset
            annotations = sorted(annotations, key=lambda x: x['start'])
            data[file_id]["annotations"] = annotations

NameError: name 'ann_dir' is not defined

## Step 3: Checking if there are words with multi labels

In [199]:
multi_label_words_count = 0
multi_label_files_count = 0
for file_id in data:
    annotations = data[file_id]['annotations']
    overlapping = False

    for i in range(len(annotations)-1):
        if annotations[i]['end'] >= annotations[i+1]['start']:
            overlapping = True
            multi_label_words_count += 1
            # print(f"Overlap found between {annotations[i]} and {annotations[i+1]}")

    if overlapping:
        multi_label_files_count += 1
        
print(f"Total number of words with multi labels found is {multi_label_words_count}")
print(f"Total number of files with multi labelled words is {multi_label_files_count}")

Total number of words with multi labels found is 809
Total number of files with multi labelled words is 105


## Checking if subwords overlaps

In [204]:
def remove_trailing_punctuation(token):
    while token and re.search(r'[^\w\s\']', token[-1]):
        token = token[:-1]
        
    return token

for file_id in data:
    annotations = data[file_id]['annotations']
    overlapping = False

    for i in range(len(annotations)-1):
        if annotations[i]['end'] > annotations[i+1]['start']:
            overlapping = True
            word_1 = data[file_id]['text'][annotations[i]['start']:annotations[i]['end']]
            word_2 = data[file_id]['text'][annotations[i+1]['start']:annotations[i+1]['end']]

            if remove_trailing_punctuation(word_1) != remove_trailing_punctuation(word_2):
                print(f"words: {word_1} {word_2}", end=" | ")
                print(f"labels: {file_id} {annotations[i]['label']} {annotations[i]['start']} {annotations[i]['end']}, {annotations[i+1]['label']} {annotations[i+1]['start']} {annotations[i+1]['end']}")
#                 print(re.sub(r'[^\w\s\']', '', word_1), re.sub(r'[^\w\s\']', '', word_2))
                

words: long-term ong-term | labels: 19214295 Detailed_description 424 433, History 425 433
words: (LV) LV | labels: 23468586 History 314 318, Disease_disorder 315 317
words: steroid steroid-induced | labels: 25155594 Medication 384 391, History 384 399
words: (HBV) HBV | labels: 27749582 History 253 258, Detailed_description 254 257


In [223]:
fix = {
    '19214295': {
            425: 424
    }
}
for file_id in data:
    if file_id in fix:
        for ann in data[file_id]['annotations']:
            if ann['start'] in fix[file_id]:
                ann['start'] = fix[file_id][ann['start']]

In [224]:
def remove_trailing_punctuation(token):
    while token and re.search(r'[^\w\s\']', token[-1]):
        token = token[:-1]
        
    return token

for file_id in data:
    annotations = data[file_id]['annotations']
    overlapping = False

    for i in range(len(annotations)-1):
        if annotations[i]['end'] > annotations[i+1]['start']:
            overlapping = True
            word_1 = data[file_id]['text'][annotations[i]['start']:annotations[i]['end']]
            word_2 = data[file_id]['text'][annotations[i+1]['start']:annotations[i+1]['end']]

            if remove_trailing_punctuation(word_1) != remove_trailing_punctuation(word_2):
                print(f"words: {word_1} {word_2}", end=" | ")
                print(f"labels: {file_id} {annotations[i]['label']} {annotations[i]['start']} {annotations[i]['end']}, {annotations[i+1]['label']} {annotations[i+1]['start']} {annotations[i+1]['end']}")
                

words: (LV) LV | labels: 23468586 History 314 318, Disease_disorder 315 317
words: steroid steroid-induced | labels: 25155594 Medication 384 391, History 384 399
words: (HBV) HBV | labels: 27749582 History 253 258, Detailed_description 254 257


In [217]:
count = 0
def remove_trailing_punctuation(token):
    while token and re.search(r'[^\w\s\']', token[-1]):
        token = token[:-1]
        
    return token

for file_id in data:
    annotations = data[file_id]['annotations']
    overlapping = False

    for i in range(len(annotations)):
        word = data[file_id]['text'][annotations[i]['start']:annotations[i]['end']]
        if '-' in word:
            count += 1
            print(word)
count 

24-year-old
non-smoking
non-bleeding
25-year-old
left-sided
intra-
follicle-stimulating
thyroid-stimulating
T1-weighted
T2-weighted
4-week
chromogranin-A
S-100
bcl-2
Ki-67
PRL-producing
69-year-old
CPK-MB
7-year-old
beta-glucosidase
carbohydrate-containing
angiotensin-converting
follow-up
follow-up
53-year-old
2-week
T-wave
R-wave
x-ray
B-type
C-reactive
anti-neutrophil
immuno-electrophoresis
M-protein
Bence-Jones
bi-atrial
Doppler-derived
trans-mitral
angiotensin-converting
low-dose
low-dose
β-blocker
dip-and-plateau
apple-green
λ-chain
β-blocker
followed-up
6-month
54-year-old
non-small
Twenty-eight
PCR-based
high-dose
whole-brain
intra-thoracic
4-year-old
bubble-like
soft-tissue
75-year-old
HBs-Ag
well-circumscribed
thoracoscopic-assisted
grayish-white
cytoplasm-rich
mid-sized
centrocyte-like
monocyte-like
B-cell
(Bcl)-2
Bcl-6
virus-encoded
follow-up
58-year-old
ST-elevation
X-rays
low-molecular-weight
non-improvement
57-year-old
short-term
contrast-enhanced
peri-rectal
sigmoid-anus

1440

Physical attributes: Mass, Height, Weight, Volume, Texture, Shape, Color

Biological concepts: Biological_attribute, Sex, Age, Disease_disorder, Biological_structure, Lab_value, Sign_symptom

Events and outcomes: Outcome, Other_event, Clinical_event, Family_history, History

Concepts related to time: Frequency, Time, Duration, Date

Concepts related to actions: Activity, Administration, Dosage, Medication, Therapeutic_procedure, Diagnostic_procedure

Other concepts: Personal_background, Subject, Area, Distance, Other_entity, Severity, Coreference, 

Nonbiological_location, Detailed_description

In [318]:
data['25884600']['text']

'A 36-yr-old previously healthy Sri Lankan male who takes care of a horse presented to the medical casualty ward with fever, arthralgia and myalgia for one day.\nHe complained of mild dysuria but had normal urine output.\nHe did not have chest pain or shortness of breath.\nFurther inquiry revealed that he was treated for leptospirosis during a febrile illness in the past.\nOn examination, patient was afebrile, anicteric.\nHis blood pressure was 90/60 mmHg and pulse rate 76 bpm.\nRest of the examination was unremarkable.\nInitial investigations performed on the day of admission revealed neutrophil leukocytosis, mild thrombocytopaenia and microscopic haematuria (Table 1).\nRenal function and liver enzymes were within normal limits.\nUrine analysis revealed pus cells 05 – 06 /hpf, red cells 30 – 35 /hpf.\nCRP (C-reactive protein) was 75 mg/dl.\nA clinical diagnosis of leptospirosis was made and he was started on intravenous penicillin in addition to adequate hydration.\nOn the second day 

To group similar tags in the NER task medical text, you can start by identifying the ones that are related. 


For example, tags such as "Disease_disorder", "Sign_symptom", "Lab_value", and "Diagnostic_procedure" are all related to medical conditions and can be grouped under a single category such as "Medical Condition". 


Similarly, tags such as "Dosage", "Medication", and "Therapeutic_procedure" can be grouped under a single category such as "Treatment". 


You can also group tags that are related to patient attributes such as "Height", "Weight", "Age", and "Sex" under a single category such as "Patient Attributes".



After grouping the tags, you can use the new categories to perform NER tasks and extract information from medical text.

If you want to remove some of the tags that are not relevant for your task, you can simply exclude them from your training data or exclude them during the evaluation of your NER model. For example, tags such as "Coreference" and "Detailed_description" may not be relevant for certain NER tasks and can be excluded.

Keep in mind that the tags you choose to keep or remove should be based on your specific task and the requirements of your NER model.



In [ ]:
{'Mass': 4,
 'Height': 'Biological_measurement',
 'Weight': 'Biological_measurement',
 'Biological_attribute': 'Biological_measurement',
 'Occupation': 41,
 'Outcome': 54,
 'Qualitative_concept': 55,
 'Quantitative_concept': 62,
 'Color': 'Biological_characteristic',
 'Personal_background': 71,
 'Volume': 71,
 'Texture': 73,
 'Subject': 'Biological_entity',
 'Shape': 'Biological_characteristic',
 'Other_event': 'Biological_entity',
 'Area': 'Biological_measurement',
 'Frequency': 'Biological_measurement',
 'Time': 'Biological_measurement',
 'Activity': 'Biological_event',
 'Sex': 'Biological_characteristic',
 'Administration': 'Biological_process',
 'Age': 253,
 'Distance': 'Biological_measurement',
 'Other_entity': 'Biological_entity',
 'Severity': 'Biological_measurement',
 'Coreference': 'Biological_entity',
 'Family_history': 'Biological_entity',
 'Clinical_event': 'Biological_event',
 'Nonbiological_location': 692,
 'Duration': 702,
 'Dosage': 856,
 'Medication': 1363,
 'Therapeutic_procedure': 1563,
 'History': 1812,
 'Date': 2058,
 'Disease_disorder': 2177,
 'Lab_value': 4625,
 'Sign_symptom': 4759,
 'Detailed_description': 4934,
 'Biological_structure': 5434,
 'Diagnostic_procedure': 7977
}

In [ ]:
# sed -i 's/Mass/Biological_measurement/g;
# s/Height/Biological_measurement/g;
# s/Weight/Biological_measurement/g;
# s/Biological_attribute/Biological_measurement/g;
# s/Color/Biological_characteristic/g;
# s/Texture/Biological_characteristic/g;
# s/Subject/Biological_entity/g;
# s/Shape/Biological_characteristic/g;
# s/Other_event/Biological_entity/g;
# s/Area/Biological_measurement/g;
# s/Frequency/Biological_measurement/g;
# s/Time/Biological_measurement/g;
# s/Activity/Biological_event/g;
# s/Sex/Biological_characteristic/g;
# s/Administration/Biological_process/g;
# s/Distance/Biological_measurement/g;
# s/Other_entity/Biological_entity/g;
# s/Severity/Biological_measurement/g;
# s/Coreference/Biological_entity/g;
# s/Family_history/Biological_entity/g;
# s/Clinical_event/Biological_event/g;
# s/Nonbiological_location/Biological_location/g;
# s/Duration/Biological_measurement/g;
# s/Dosage/Biological_measurement/g;
# s/Medication/Biological_entity/g;
# s/Therapeutic_procedure/Biological_process/g;
# s/History/Biological_entity/g;
# s/Date/Biological_measurement/g;
# s/Disease_disorder/Disease/g;
# s/Lab_value/Biological_measurement/g;
# s/Sign_symptom/Symptom/g;
# s/Detailed_description/Biological_measurement/g;
# s/Biological_structure/Biological_entity/g;
# s/Diagnostic_procedure/Biological_process/g' file.bio


In [ ]:
# sed -i 's/Occupation/O/g; s/Outcome/O/g; s/Qualitative_concept/O/g; s/Quantitative_concept/O/g; s/Personal_background/O/g' *.bio

In [127]:
count=0
label_count = {}
for file_id in data:
    annotations = data[file_id]['annotations']
    for annotation in annotations:
        if annotation['label'] not in label_count:
            label_count[annotation['label']] = 0
        label_count[annotation['label']] += 1


dict(sorted(label_count.items(), key=lambda x: x[1]))


{'Mass': 4,
 'Height': 8,
 'Weight': 8,
 'Biological_attribute': 13,
 'Occupation': 41,
 'Outcome': 54,
 'Qualitative_concept': 55,
 'Quantitative_concept': 62,
 'Color': 68,
 'Personal_background': 71,
 'Volume': 71,
 'Texture': 73,
 'Subject': 77,
 'Shape': 82,
 'Other_event': 89,
 'Area': 139,
 'Frequency': 159,
 'Time': 164,
 'Activity': 176,
 'Sex': 191,
 'Administration': 202,
 'Age': 253,
 'Distance': 274,
 'Other_entity': 323,
 'Severity': 401,
 'Coreference': 436,
 'Family_history': 486,
 'Clinical_event': 683,
 'Nonbiological_location': 692,
 'Duration': 702,
 'Dosage': 856,
 'Medication': 1363,
 'Therapeutic_procedure': 1563,
 'History': 1812,
 'Date': 2058,
 'Disease_disorder': 2177,
 'Lab_value': 4625,
 'Sign_symptom': 4759,
 'Detailed_description': 4934,
 'Biological_structure': 5434,
 'Diagnostic_procedure': 7977}

In [339]:
label_2_words = {}
for file_id in data:
    text = data[file_id]['text']
    annotations = data[file_id]['annotations']
    
    for ann in annotations:
        label = ann['label']
        start = ann['start']
        end = ann['end']
        
        if label not in label_2_words:
            label_2_words[label] = set()
        label_2_words[label].add(text[start:end])

In [341]:
label_2_words['Disease_disorder']

{'(CHF)',
 '(LV)',
 '(PID)',
 '(carcinoid)',
 '(serotype',
 '2',
 '3',
 'A',
 'A)',
 'ACTH',
 'ADC',
 'ADEM',
 'ADH',
 'ADHD',
 'AE',
 'AFLP',
 'AKI',
 'AL',
 'AOA',
 'AOSD',
 'ARDS',
 'ARVC',
 'ASD',
 'ATM',
 'AVED',
 'AVM',
 'Abrikossoff',
 'Anti',
 'Aspergillus',
 'AstV',
 'Ataxia',
 'Atresia',
 'Atrial',
 'B',
 'B1',
 'BCC',
 'BCS',
 'Balo',
 'Barr',
 'Barré',
 'Barth',
 'Blau',
 'Bowen’s',
 'Burkitt',
 'C',
 'CAD',
 'CAPD',
 'CC',
 'CD',
 'CF',
 'CHB',
 'CHF',
 'CML',
 'CMP',
 'CMT',
 'CNS',
 'CNV',
 'CO2',
 'COPD',
 'CVID',
 'Cardiac',
 'Castleman',
 'Charcot',
 'Charlevoix',
 'Cognard',
 'Constrictive',
 'Cor',
 'Cowden',
 "Crohn's",
 'Cryptococcus',
 'DAH',
 'DCIS',
 'DCM',
 'DD',
 'DIC',
 'DKA',
 'DLBCL',
 'DVAF',
 'DVT',
 'Danlos',
 'Deep',
 'Drug',
 'E',
 'EAS',
 'ECD',
 'ECF',
 'ESRD',
 "Ebstein's",
 'Ehlers',
 'Enterococcus',
 'Epididymitis',
 'Epstein',
 'Escherichia',
 'FRDA',
 'Fanconi',
 "Friedreich's",
 'G6PD',
 'GCC',
 'GD',
 'GHD',
 'GIST',
 'GPA',
 'GVHD',
 'Gauche

In [152]:
dict(sorted(label_count.items(), key=lambda x: x[1]))

{'Mass': 4,
 'Height': 8,
 'Weight': 8,
 'Biological_attribute': 13,
 'Occupation': 41,
 'Outcome': 54,
 'Qualitative_concept': 55,
 'Quantitative_concept': 62,
 'Color': 68,
 'Personal_background': 71,
 'Volume': 71,
 'Texture': 73,
 'Subject': 77,
 'Shape': 82,
 'Other_event': 89,
 'Area': 139,
 'Frequency': 159,
 'Time': 164,
 'Activity': 176,
 'Sex': 191,
 'Administration': 202,
 'Age': 253,
 'Distance': 274,
 'Other_entity': 323,
 'Severity': 401,
 'Coreference': 436,
 'Family_history': 486,
 'Clinical_event': 683,
 'Nonbiological_location': 692,
 'Duration': 702,
 'Dosage': 856,
 'Medication': 1363,
 'Therapeutic_procedure': 1563,
 'History': 1812,
 'Date': 2058,
 'Disease_disorder': 2177,
 'Lab_value': 4625,
 'Sign_symptom': 4759,
 'Detailed_description': 4934,
 'Biological_structure': 5434,
 'Diagnostic_procedure': 7977}

In [226]:
data

{'19860925': {'text': 'Our 24-year-old non-smoking male patient presented with repeated hemoptysis in May 2008 with 4 days of concomitant right thoracic pain which intensified while breathing.\nDuring holidays in his home country, this Cuban patient suffered from a cold with fever and a strong cough.\nThe strong dry cough persisted after recovery from the cold.\nThe patient did not report any loss of weight.\nThe initial CT scan of the thorax showed a 12 × 4 cm solid mass paravertebral right in the lower thorax without any signs of metastases (Figure 1).\nThe bronchoscopy (Figure \u200b2) with non-bleeding biopsy revealed a mass of the lower right bronchus which histologically and immunohistologically provided evidence of a granular cell or Abrikossoff tumor [1].\nThe bronchial lavage which followed was negative for malignant cells.\nThe patient was discharged and surgical intervention was planned.\nFour days after discharge a spontaneous hemothorax developed.\nThe patient needed to be

In [157]:
for tag in [
    'Clinical_event',
    'Sign_symptom',
    'Biological_structure',
    'Diagnostic_procedure',
    'Disease_disorder',
    'Lab_value',
    'Therapeutic_procedure',
    'Medication',
    'Family_history'
]:
    print(tag, len(label_2_words[tag]))

Clinical_event 153
Sign_symptom 1610
Biological_structure 1204
Diagnostic_procedure 2203
Disease_disorder 919
Lab_value 1969
Therapeutic_procedure 652
Medication 612
Family_history 236


Group 'Outcome', 'Severity', and 'Sign_symptom' into a single 'Symptom' category, as they all describe aspects of a patient's medical condition.

Group 'Family_history', 'Clinical_event', and 'History' into a single 'Medical history' category, as they all pertain to a patient's past medical events and conditions.

Group 'Biological_structure' and 'Diagnostic_procedure' into a single 'Medical procedure' category, as they both involve medical tests and procedures.

Keep 'Date' and 'Duration' as separate categories, as they pertain to temporal aspects of medical events.

1. Group 'Mass', 'Height', 'Weight', 'Volume', 'Area', 'Distance' into a single 'Measurement' category.
2. Group 'Qualitative_concept' and 'Quantitative_concept' into a single 'Concept' category.
3. Group 'Texture' and 'Shape' into a single 'Appearance' category.
4. Group 'Outcome', 'Severity', 'Coreference', 'Family_history', 'Clinical_event', 'Nonbiological_location', 'Duration', 'History', 'Date', 'Sign_symptom', 'Detailed_description', 'Biological_structure', 'Diagnostic_procedure', and 'Disease_disorder' into a single 'Condition' category.

After grouping the tags, you can assign a shorthand to each category, such as 
* 'MEAS' for 'Measurement', 
* 'CONC' for 'Concept', 
* 'APP' for 'Appearance', and 
* 'COND' for 'Condition'. 

This will reduce the number of tags you need to consider and simplify your NER task, while still covering a wide range of medical concepts.

In [144]:
label_2_words['Detailed_description']

{'Scientific,',
 'mycobacteria',
 'Hi-Torque',
 'intracavitary',
 'breathing',
 '2nd',
 'unclear',
 'short-term',
 'pork,',
 'supraperichondrial',
 'pulsatile',
 'iron',
 'sequence',
 'None',
 'cords',
 'thoracoscopic',
 'exercise',
 'origin',
 'nearby',
 'T-50,',
 'DAH',
 'GRCh37/hg19',
 'fluoroscopic',
 'slightly',
 'a',
 'arms',
 'cycle',
 'occasion',
 'KCNQ1',
 'oil',
 'markers',
 'Whole-exome',
 'smoking',
 'pegylated',
 'fast',
 '10\u2009min',
 'ePTFE',
 'image',
 'NM_000218.2:c.115G\u2009>\u2009T',
 'Rapid',
 'Autologous',
 'hard',
 'traditional',
 'Twelve-lead',
 'pigmented',
 'fluids',
 'on',
 'PCR-based',
 'EGFR',
 'tonico-clonic',
 'mobile',
 'showed',
 'Corporation,',
 'asymmetry',
 'WISC-R',
 'air',
 'symmetrical',
 '24-mm',
 'postmenopausal',
 'dry',
 'Endurant',
 'antioxidant',
 'ARDS',
 'radiation-induced',
 '3.0®',
 'recipients',
 'perigastric',
 'Echolaser',
 '(ResMed)',
 '5\u200aW',
 'Systems;',
 'caudal',
 'clamped',
 'cardiac',
 'EU-ME2',
 'delay',
 'vasculitis',
 

In [132]:
entities = [
    'Disease_disorder',
    'Sign_symptom',
    'Therapeutic_procedure',
    'Medication',
    'Dosage',
    'Lab_value',
    'Diagnostic_procedure'
]
for entity in entities:
    print(entity, label_count[entity])

Disease_disorder 2177
Sign_symptom 4759
Therapeutic_procedure 1563
Medication 1363
Dosage 856
Lab_value 4625
Diagnostic_procedure 7977


In [ ]:
   
# Step 3: Convert annotations to .bio format
bio_data = []
for doc in data:
    tokens = doc["text"].split()
    bio_tokens = []
    for i, token in enumerate(tokens):
        label = "O"
        for ann in doc["annotations"]:
            if i >= ann["start"] and i < ann["end"]:
                if i == ann["start"]:
                    label = "B-" + ann["label"]
                else:
                    label = "I-" + ann["label"]
        bio_tokens.append((token, label))
    bio_data.append(bio_tokens)
    
# Step 4: Split data into train, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_docs = len(bio_data)
train_end = int(train_ratio * num_docs)
val_end = train_end + int(val_ratio * num_docs)